# Question 2(b)
Use an LLM to generate predictions see section 3 of [LLMTIME](https://arxiv.org/abs/2310.07820) for some ideas on
tokenisation of input, handling null values. Report only your best working prompting strategy
and their evaluations. (You are not expected to train/finetune an LLM nor use multimodal LLM
i.e you input pdf or screenshot, your input to the LLM should be PROMPT + string generated
from timeseries dataframe ideally by a script)

In [ ]:
import pandas as pd

Let's clean the data first.

In [ ]:
data = pd.read_csv("AirtrafficA4.csv")
data['PASSENGERS CARRIED'] = data['PASSENGERS CARRIED'].str.replace(",", "").astype(float)
data.fillna(float(0), inplace=True)
data

Then generate the prompt

In [36]:
def format_for_prompt(row):
    return f"Airline {row['AIRLINE']}, Year {row['YEAR']}, Month {row['MONTH']}, Passengers Carried: {row['PASSENGERS CARRIED']}."

prompt_data = " ".join(data.apply(format_for_prompt, axis = 1).tolist())
prompt_data = " Given the following monthly airline passenger data, predict and display the values for next 12 months (2023 SEP to 2024 AUG) for Passengers Carried, you need not show any code or your though process, just the final predicted values must be displayed, there's a season of 4 months: " + prompt_data
prompt_data

" Given the following monthly airline passenger data, predict and display the values for next 12 months (2023 SEP to 2024 AUG) for Passengers Carried, you need not show any code or your though process, just the final predicted values must be displayed, there's a season of 4 months: Airline A007, Year 2023, Month JAN, Passengers Carried: 6847384.0. Airline A007, Year 2023, Month FEB, Passengers Carried: 6741948.0. Airline A007, Year 2023, Month MAR, Passengers Carried: 7317288.0. Airline A007, Year 2023, Month APR, Passengers Carried: 7406440.0. Airline A007, Year 2023, Month MAY, Passengers Carried: 8109626.0. Airline A007, Year 2023, Month JUNE, Passengers Carried: 7893296.0. Airline A007, Year 2023, Month JULY, Passengers Carried: 7674890.0. Airline A007, Year 2023, Month AUG, Passengers Carried: 7867259.0. Airline A007, Year 2022, Month JAN, Passengers Carried: 3556558.0. Airline A007, Year 2022, Month FEB, Passengers Carried: 3950597.0. Airline A007, Year 2022, Month MAR, Passenger

Here's the output predicted by GPT-4o:

|YEAR_MONTH|PASSENGERS CARRIED|
|----------|------------------|
|2023 SEP|7789024|
|2023 OCT|8122317|
|2023 NOV|8225536|
|2023 DEC|8504783|
|2024 JAN|7739815|
|2024 FEB|7631294|
|2024 MAR|8124572|
|2024 APR|8218501|
|2024 MAY|8802926|
|2024 JUN|8550792|
|2024 JUL|8289312|
|2024 AUG|8494387|

This gave a score of `2.50` on the Kaggle competition